In [2]:
## This file implements neural networks and random forest on pyopresabsSTCC_qual.
## Due to the imbalanced data and limited cases for class 2, we implement the over-sampling method.
## For fully-connected neural networks, the accuracy is 97.46% for over-sampling data, and 97.97% after regularization.
## For random forest, the accuracy is 98.48% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 96.05% for over-sampling.

In [3]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [4]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/pyopresabsSTCC_qual.csv')
df.shape

(255, 541)

In [5]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [6]:
df['pheno']

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     1
25     0
26     0
27     0
28     0
29     0
      ..
225    0
226    0
227    1
228    1
229    1
230    0
231    0
232    0
233    0
234    0
235    0
236    0
237    0
238    0
239    0
240    0
241    0
242    0
243    0
244    0
245    0
246    0
247    0
248    0
249    0
250    0
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [7]:
df['pheno'].value_counts()

0    218
1     33
2      4
Name: pheno, dtype: int64

In [8]:
df.head()

,id,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,...,group_8836,group_8913,group_9026,group_9048,group_9123,group_9129,group_9226,ST,CC,pheno
0,107,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,5,5,0
1,109,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,0
2,115,0,1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,5,5,0
3,120335,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,5,5,0
4,120337,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,5,5,0


In [9]:
df_clean = df.drop(columns=['id'])

In [10]:
df_clean.shape

(255, 540)

In [11]:
df_clean.head()

,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,GAACCATGGACATCATGTGAATTTGATTTTACAAGAGAGGGT,...,group_8836,group_8913,group_9026,group_9048,group_9123,group_9129,group_9226,ST,CC,pheno
0,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,5,5,0
1,0,0,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,8,8,0
2,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,5,5,0
3,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,5,5,0
4,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,5,5,0


In [12]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 539) (255,)


In [13]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 218), (1, 218), (2, 218)]


Using TensorFlow backend.


In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
############# Fully-Connected Neural Network ################

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [17]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [18]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 457 samples, validate on 197 samples
Epoch 1/100
457/457 [==============================] - 0s 501us/step - loss: 1.3334 - accuracy: 0.5492 - val_loss: 0.7934 - val_accuracy: 0.6244
Epoch 2/100
457/457 [==============================] - 0s 123us/step - loss: 0.6404 - accuracy: 0.7330 - val_loss: 0.5608 - val_accuracy: 0.7868
Epoch 3/100
457/457 [==============================] - 0s 174us/step - loss: 0.4672 - accuracy: 0.8468 - val_loss: 0.4370 - val_accuracy: 0.8325
Epoch 4/100
457/457 [==============================] - 0s 113us/step - loss: 0.3604 - accuracy: 0.8928 - val_loss: 0.3655 - val_accuracy: 0.8731
Epoch 5/100
457/457 [==============================] - 0s 101us/step - loss: 0.3008 - accuracy: 0.9147 - val_loss: 0.3380 - val_accuracy: 0.8782
Epoch 6/100
457/457 [==============================] - 0s 97us/step - loss: 0.2607 - accuracy: 0.9387 - val_loss: 0.2820 - val_accuracy: 0.8985
Epoch 7/100
457/457 [==============================] - 0s 116us/step - loss: 0.2307 -

Epoch 57/100
457/457 [==============================] - 0s 91us/step - loss: 0.0088 - accuracy: 1.0000 - val_loss: 0.0689 - val_accuracy: 0.9746
Epoch 58/100
457/457 [==============================] - 0s 99us/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 0.0763 - val_accuracy: 0.9746
Epoch 59/100
457/457 [==============================] - 0s 92us/step - loss: 0.0076 - accuracy: 1.0000 - val_loss: 0.0721 - val_accuracy: 0.9746
Epoch 60/100
457/457 [==============================] - 0s 142us/step - loss: 0.0074 - accuracy: 1.0000 - val_loss: 0.0726 - val_accuracy: 0.9746
Epoch 61/100
457/457 [==============================] - 0s 120us/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 0.0716 - val_accuracy: 0.9746
Epoch 62/100
457/457 [==============================] - 0s 106us/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 0.0722 - val_accuracy: 0.9746
Epoch 63/100
457/457 [==============================] - 0s 94us/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 0.0720 - val

In [20]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

197/197 [==============================] - 0s 90us/step
over-sampling test accuracy: 97.46%


In [21]:
#### add dropout and regularizor
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [22]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 457 samples, validate on 197 samples
Epoch 1/100
457/457 [==============================] - 0s 571us/step - loss: 11.9220 - accuracy: 0.3895 - val_loss: 7.3780 - val_accuracy: 0.5178
Epoch 2/100
457/457 [==============================] - 0s 72us/step - loss: 9.1279 - accuracy: 0.4770 - val_loss: 3.9003 - val_accuracy: 0.6497
Epoch 3/100
457/457 [==============================] - 0s 71us/step - loss: 6.8365 - accuracy: 0.4989 - val_loss: 3.8276 - val_accuracy: 0.6396
Epoch 4/100
457/457 [==============================] - 0s 76us/step - loss: 5.8987 - accuracy: 0.5274 - val_loss: 2.7865 - val_accuracy: 0.6904
Epoch 5/100
457/457 [==============================] - 0s 76us/step - loss: 4.8229 - accuracy: 0.5930 - val_loss: 1.8852 - val_accuracy: 0.7310
Epoch 6/100
457/457 [==============================] - 0s 78us/step - loss: 4.4707 - accuracy: 0.5602 - val_loss: 1.3733 - val_accuracy: 0.7411
Epoch 7/100
457/457 [==============================] - 0s 112us/step - loss: 3.0391 - ac

457/457 [==============================] - 0s 73us/step - loss: 1.2667 - accuracy: 0.8096 - val_loss: 0.6175 - val_accuracy: 0.9645
Epoch 58/100
457/457 [==============================] - 0s 68us/step - loss: 1.1957 - accuracy: 0.7877 - val_loss: 0.6281 - val_accuracy: 0.9645
Epoch 59/100
457/457 [==============================] - 0s 73us/step - loss: 1.3542 - accuracy: 0.7899 - val_loss: 0.6521 - val_accuracy: 0.9492
Epoch 60/100
457/457 [==============================] - 0s 68us/step - loss: 1.2075 - accuracy: 0.7834 - val_loss: 0.6418 - val_accuracy: 0.9695
Epoch 61/100
457/457 [==============================] - 0s 66us/step - loss: 1.0681 - accuracy: 0.8074 - val_loss: 0.6495 - val_accuracy: 0.9746
Epoch 62/100
457/457 [==============================] - 0s 69us/step - loss: 1.2489 - accuracy: 0.7812 - val_loss: 0.6566 - val_accuracy: 0.9543
Epoch 63/100
457/457 [==============================] - 0s 71us/step - loss: 1.0110 - accuracy: 0.8074 - val_loss: 0.6337 - val_accuracy: 0.969

In [24]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

197/197 [==============================] - 0s 73us/step
over-sampling test accuracy: 97.97%


In [25]:
############## Random Forest ##############

In [26]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [27]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 98.48%


In [28]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.95698925 0.97849462 0.96703297 0.94444444 0.95555556]
0.9605033676001419
